# Tiny A11y LoRA Chatbot
This notebook loads your LoRA fine-tuned `younglim/tiny-a11y-model` on DeepSeek-Coder and lets you chat interactively.

In [ ]:
# Install required packages
!pip install --upgrade pip
!pip install transformers accelerate peft gradio

In [ ]:
# Load base model and tokenizer (FP16 + device_map for GPU)
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModelForCausalLM
import torch

base_model_name = 'deepseek-ai/deepseek-coder-1.3b-instruct'
lora_model_name = 'younglim/tiny-a11y-model'

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Load base model in FP16 on GPU
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map='auto',        # automatically place layers on GPU
    torch_dtype=torch.float16 # use half-precision for speed
)

# Load LoRA adapter
model = PeftModelForCausalLM.from_pretrained(base_model, lora_model_name)

# Ensure model is in eval mode
model.eval()

In [ ]:
# Chat function using optimized PyTorch generate
def chat(prompt, max_new_tokens=200):
    # Tokenize input and move to model device
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

    # Generate output
    with torch.no_grad():  # disable gradients for inference
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)



In [ ]:
# Launch Gradio interface
import gradio as gr

iface = gr.Interface(
    fn=chat,
    inputs=gr.Textbox(lines=5, label='Enter your prompt'),
    outputs=gr.Textbox(lines=15, label='Model response'),
    title='Tiny A11y Chatbot',
    description='Ask the Tiny A11y Model anything about accessibility.'
)

# Set share=True to get a public link
iface.launch(share=True)